In [1]:
import pandas
from pathlib import Path
from pprint import pprint
from io import BytesIO
import re
import sys
import tarfile
from collections import Counter, namedtuple

In [2]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)
from encoded_client.encoded import ENCODED, HTTPError

In [3]:
known_document_types = {
    '/documents/ecb1c489-905b-4afb-8389-ed7fb2e36ff2/': "Chromium v2",
    '/documents/515a990c-50c4-459a-8036-502feaf5a18c/': "Chromium v2 technical note",
    '/documents/3408b7a1-6b1b-45d1-a73e-db0556e6abdc/': "Split-seq analysis protocol",
    '/documents/3cdf327b-df85-4149-9fca-a7541e6f1cb1/': "Split-seq overview",
    '/documents/c155e58a-13ab-4af9-8591-cee13d639a40/': "Nuclei Isolation protocol from tissue for 10x multiome",
    '/documents/53fb4382-c54f-4761-89bb-6a01a0dca97d/': "Chromium GEM v3.1",
    '/documents/4d42b1ef-c9e6-4b31-a443-1266079545d0/': "Chromium Multiome ATAC + Gene Expression",
    '/documents/d18bad0e-9a86-40f0-8ded-2c74072910e5/': "Nuclei Isolation protocol from tissue for 10x multiome",  # probably same as c155e58a-13ab-4af9-8591-cee13d639a40
    '/documents/e3110414-4013-425e-94f6-72c549702bde/': "Pool split protocol",
    '/documents/1bb75b62-ac29-4368-9855-68d410e1963a/': "GM12878 Cell Culture protocol",
    '/documents/28fef8e0-e171-450d-b66d-e3d199531cf2/': "Ambion spike in mixture",
    '/documents/a6e06058-5019-488f-9195-8bcd2e9229c1/': "Laser capture microdisection",
    '/documents/c4b8952c-d9f1-42e7-8ea5-54659d14f46b/': "Human tissue preservation protocol flash-freezing (version 1)",
    '/documents/c5c4174c-0f33-4793-be9d-f2b6a9357dae/': "Nuclei Isolation protocol from tissue for 10x multiome", # probably same as c155e58a-13ab-4af9-8591-cee13d639a40
    '/documents/84a66218-cd6a-4959-adc0-91ecc513117f/': "Chromium Multiome ATAC + Gene Expression",
    '/documents/c6235c97-a978-41b6-8a7e-1e16b983f4ba/': "inDrop Sample Collection Protocol",
    '/documents/8179222b-9ad7-4274-a07e-820dbcac6aca/': "inDrop Library Prep",
    '/documents/80953249-5d87-4f2c-84e0-0212c404c8d6/': "Nuclei Isolation protocol from tissue 10x snRNA", # probably same as c155e58a-13ab-4af9-8591-cee13d639a40
    '/documents/57177fd0-da98-4e8f-95a4-459fbaeea146/': "Chromium GEM v3.1", # psame as 53fb4382-c54f-4761-89bb-6a01a0dca97d
    '/documents/5f5193b0-0c61-40c6-a286-b1f6d6f0c0da/': "SmartSeq",
    '/documents/9a7e281f-b901-422b-9f01-378ba4b9e97f/': "Mouse adrenal gland nuclei isolation protocol for single nucleus RNA-seq",
    '/documents/f1b60b4e-b5d2-4b8d-af0b-06b767fa07c3/': "Biosample purification protocol",
    '/documents/49fd5b3b-9878-4ec8-92bf-ef6dce9116e3/': "SMARTer Ultra Low RNA kit",
    "/documents/f16e44c9-ee9d-4ede-9bb5-886ea451d7a0/": "Cortex nuclei isolation protocol",
}


In [4]:
document_to_protocol = {
    '/documents/ecb1c489-905b-4afb-8389-ed7fb2e36ff2/': "Chromium v2",
    #'/documents/515a990c-50c4-459a-8036-502feaf5a18c/': "Chromium v2",
    '/documents/3408b7a1-6b1b-45d1-a73e-db0556e6abdc/': "Split-seq",
    '/documents/3cdf327b-df85-4149-9fca-a7541e6f1cb1/': "Split-seq",
    #'/documents/c155e58a-13ab-4af9-8591-cee13d639a40/': "Nuclei Isolation protocol from tissue for 10x multiome",
    '/documents/53fb4382-c54f-4761-89bb-6a01a0dca97d/': "Chromium GEM v3.1",
    '/documents/4d42b1ef-c9e6-4b31-a443-1266079545d0/': "Chromium Multiome ATAC + Gene Expression", # same as 53fb4382-c54f-4761-89bb-6a01a0dca97d
    #'/documents/d18bad0e-9a86-40f0-8ded-2c74072910e5/': "Nuclei Isolation protocol from tissue for 10x multiome",  # probably same as c155e58a-13ab-4af9-8591-cee13d639a40
    '/documents/d18bad0e-9a86-40f0-8ded-2c74072910e5/': "Chromium Multiome ATAC + Gene Expression",
    '/documents/e3110414-4013-425e-94f6-72c549702bde/': "SmartSeq Pool",
    '/documents/a6e06058-5019-488f-9195-8bcd2e9229c1/': "SmartSeq",
    #'/documents/c5c4174c-0f33-4793-be9d-f2b6a9357dae/': "Nuclei Isolation protocol from tissue for 10x multiome", # probably same as c155e58a-13ab-4af9-8591-cee13d639a40
    '/documents/84a66218-cd6a-4959-adc0-91ecc513117f/': "Chromium Multiome ATAC + Gene Expression",
    '/documents/c6235c97-a978-41b6-8a7e-1e16b983f4ba/': "inDrop",
    '/documents/8179222b-9ad7-4274-a07e-820dbcac6aca/': "inDrop",
    #'/documents/80953249-5d87-4f2c-84e0-0212c404c8d6/': "Nuclei Isolation protocol from tissue 10x snRNA", # probably same as c155e58a-13ab-4af9-8591-cee13d639a40
    '/documents/57177fd0-da98-4e8f-95a4-459fbaeea146/': "Chromium GEM v3.1", # psame as 53fb4382-c54f-4761-89bb-6a01a0dca97d
    '/documents/5f5193b0-0c61-40c6-a286-b1f6d6f0c0da/': "SmartSeq",
    '/documents/49fd5b3b-9878-4ec8-92bf-ef6dce9116e3/': "SmartSeq",
}

#From alex https://en-coding.slack.com/archives/C014UJ5F73Q/p1633558205130000
#10X 3' protocol v2 and v3 were Forward-stranded, 5'-v2 is Reverse-stranded. I am not aware of any of their protocols that are unstranded.
protocol_defaults = {
    "Chromium v2": {
        "stranded": "Forward",
        "umi_length": 10,
        #"inclusion_list_url": "https://encd-6248-78c147870-qing.demo.encodedcc.org/files/737K-august-2016/@@download/737K-august-2016.txt.gz"
        #"inclusion_list_url": "https://woldlab.caltech.edu/~diane/genome/737K-arc-v1.txt.gz"
        #"inclusion_list_url": "https://encd-6248-24bd77146-qing.demo.encodedcc.org/files/737K-august-2016/@@download/737K-august-2016.txt.gz"
        "inclusion_accession": "737K-august-2016"
    },
    "Chromium GEM v3.1": {
        "stranded": "Forward",
        "umi_length": 12,
        #"inclusion_list_url": "https://encd-6248-24bd77146-qing.demo.encodedcc.org/files/3M-february-2018/@@download/3M-february-2018.txt.gz"
        "inclusion_accession": "3M-february-2018"
    },
    "Chromium Multiome ATAC + Gene Expression": {
        "stranded": "Forward",
        "umi_length": 12,
        "inclusion_accession": "737K-arc-v1(GEX)",
        #"inclusion_list_url": "https://encd-6248-78c147870-qing.demo.encodedcc.org/files/737K-arc-v1(GEX)/@@download/737K-arc-v1(GEX).txt.gz",
        #"inclusion_list_url": "https://woldlab.caltech.edu/~diane/genome/737K-arc-v1(GEX).txt.gz",
        #"inclusion_list_url": "https://encd-6248-24bd77146-qing.demo.encodedcc.org/files/737K-arc-v1(GEX)/@@download/737K-arc-v1(GEX).txt.gz"
    },
    "Split-seq" : {
        "stranded": "Forward",
        "inclusion_list_url": "https://woldlab.caltech.edu/~diane/genome/parse-splitseq-v1.tar.gz",
    }
}

default_genome_index = {
    #"/organisms/human/": "https://encd-6248-78c147870-qing.demo.encodedcc.org/files/TSTFF202658/@@download/TSTFF202658.tar.gz"
    "/organisms/human/": {
        #"genome_index_url": "https://woldlab.caltech.edu/~diane/genome/GRCh38-V29-male-2.7.8a.tar.gz",
        "genome_accession": "encode:starsolo-GRCh38-V29-male-index",
        #"assembly": "GRCh38",
        #"genome_annotation": "V29",
    },
    "/organisms/mouse/": {
        #"genome_index_url": "https://woldlab.caltech.edu/~diane/genome/mm10-M21-male-2.7.9a/mm10-M21-male-2.7.9a-star.tar.gz",
        "genome_accession": "encode:starsolo-mm10-M21-male-index",
        #"assembly": "mm10",
        #"genome_annotation": "M21",
    }
}

In [5]:
server = ENCODED("www.encodeproject.org")
#server = ENCODED("test.encodedcc.org")

In [6]:
experiment = server.get_json("/experiments/ENCSR203THX/")
has_processed = []
for f in experiment["files"]:
    submitted_file_name = Path(f["submitted_file_name"])
    if f.get("output_type") == "sparse gene count matrix of unique reads":
        print(submitted_file_name)
    elif len(f.get("analyses", [])) > 0:
        title = f["analyses"][0]["title"]
        print(f['accession'], title)


GeneFull_Ex50pAS_Unique_filtered.tar.gz


In [7]:
query = server.search_jsonld(type="Experiment", assay_title="scRNA-seq", limit="all")

In [8]:
documents_seen = Counter()
results = []
needed_metadata = []
experiment_cache = {}
matching_strand = 0
for row in query["@graph"]:
    if row["status"] in ("in progress", "released"):
        experiment = server.get_json(row["@id"])
        documents = set(experiment.get("documents", []))
        construction_method = set()
        for replicate in experiment['replicates']:
            library = replicate['library']
            library_barcode = library.get("barcode_details")
            for d in library.get("documents", []):
                documents.add(d)
            construction_platform = library.get('construction_platform', None)
            if construction_platform is not None:                
                construction_method.add(construction_platform["term_name"])
            construction_platform = library.get("construction_method", None)
            if construction_platform is not None:
                construction_method.add(construction_platform[0])

            biosample = library["biosample"]
            for d in biosample.get("documents", []):
                documents.add(d)
            subcellular_fraction = biosample.get("subcellular_fraction_term_name", "")
                
        has_processed = False
        for f in experiment["files"]:
            submitted_file_name = Path(f["submitted_file_name"])            
            if f.get("output_type") == "sparse gene count matrix of unique reads":
                has_processed = True

        sc_type = set()

        for d in documents:
            assert d in known_document_types
            if d in document_to_protocol:
                sc_type.add(document_to_protocol[d])
        if len(experiment["documents"]) > 0:
            documents_seen[",".join(experiment["documents"])] += 1
          
        if len(sc_type) == 1:
            sc_type = sc_type.pop()
        elif len(sc_type) < 1:
            sc_type = "Unknown type: {}".format(experiment["accession"], documents)
        elif len(sc_type) > 1:
            sc_type = "Mixed types: {}".format(experiment["accession"], sc_type)
        
        strand = protocol_defaults.get(sc_type, {}).get("stranded", None)
        
        if strand is not None:
            strand_specificity = library.get("strand_specificity", "not set").lower()
            if strand.lower() != strand_specificity:
                print("{} {} != {}".format(library["accession"], strand, strand_specificity))
            else:
                matching_strand += 1
        
        if (sc_type.startswith("Chromium") or sc_type.startswith("Split-seq")):
            could_process = True
        else:
            could_process = False
        
        experiment_cache[row["accession"]] = experiment
        results.append({
            "experiment": row["accession"], 
            "library": library["accession"],
            "status": row["status"],
            "could_process": "Y" if could_process else "N",
            "lab": row["lab"]["title"].split(',')[1].strip(), 
            "description": experiment.get("description", ""),
            "documents": documents,
            "document_len": len(documents),
            "construction_method": ",".join(construction_method),
            #"library_barcode": library_barcode,
            #"library_description": library.get("description", ""),
            "biosample_ontology": biosample["biosample_ontology"]["@id"],
            "biosample_term": biosample["biosample_ontology"]["term_name"],
            "sc_type": sc_type,
            "organism": biosample["organism"]["name"],
            "subcellular_fraction": subcellular_fraction,
            "has_processed": "Y" if has_processed else "N",
        })
        
print("matching_strand", matching_strand)

ENCLB523JWN Forward != unstranded
ENCLB072RWF Forward != unstranded
ENCLB885OKL Forward != unstranded
ENCLB849GNT Forward != unstranded
ENCLB581NBT Forward != unstranded
ENCLB524IGV Forward != unstranded
ENCLB010FKE Forward != unstranded
ENCLB919KOR Forward != unstranded
ENCLB989CJP Forward != unstranded
ENCLB010VHV Forward != unstranded
ENCLB716MVI Forward != unstranded
ENCLB773PPF Forward != unstranded
ENCLB902RKO Forward != unstranded
ENCLB536TLF Forward != unstranded
ENCLB374JXM Forward != unstranded
ENCLB343QQX Forward != unstranded
ENCLB129SKD Forward != unstranded
ENCLB709WAG Forward != unstranded
ENCLB692PHT Forward != unstranded
ENCLB102VRS Forward != unstranded
ENCLB169FGN Forward != unstranded
ENCLB988KOO Forward != unstranded
ENCLB848YMF Forward != unstranded
ENCLB332MGL Forward != unstranded
ENCLB323CCI Forward != unstranded
ENCLB678GYJ Forward != unstranded
ENCLB382NBT Forward != unstranded
ENCLB592ZMI Forward != unstranded
ENCLB194LYE Forward != unstranded
ENCLB735WRJ Fo

ENCLB357XZN Forward != reverse
ENCLB611BAN Forward != reverse
ENCLB025XJD Forward != unstranded
ENCLB151JZY Forward != reverse
ENCLB807HIR Forward != reverse
ENCLB866MAL Forward != reverse
ENCLB814OYP Forward != unstranded
ENCLB592LXS Forward != unstranded
ENCLB615BOP Forward != unstranded
ENCLB812SMT Forward != unstranded
ENCLB608YNH Forward != reverse
ENCLB932FNN Forward != reverse
ENCLB181TPV Forward != unstranded
ENCLB495DSX Forward != reverse
ENCLB327PYA Forward != reverse
ENCLB063JPD Forward != reverse
ENCLB871PHY Forward != reverse
ENCLB328JIP Forward != unstranded
ENCLB713ETA Forward != reverse
ENCLB676AMA Forward != reverse
ENCLB418IDT Forward != reverse
ENCLB272QLF Forward != reverse
ENCLB578ADC Forward != unstranded
ENCLB103OKD Forward != reverse
ENCLB432ZEQ Forward != reverse
ENCLB643HGB Forward != unstranded
ENCLB674XYV Forward != reverse
ENCLB909DQG Forward != unstranded
ENCLB831IHU Forward != unstranded
ENCLB619VVF Forward != unstranded
ENCLB527WWJ Forward != unstranded


In [9]:
#report_format = "{accession:12} {status:12} {lab:10} {document_len:3} {has_processed:4} {construction_method:35.35} {sc_type:20}"
#  {document_len:3}
report_format = "{experiment:12} {status:12} {lab:10} {could_process:5} {has_processed:5} {organism:8.8} {subcellular_fraction:8.8} {sc_type:20.20} {biosample_term}"
print(report_format.format(**{
    "experiment": "Experiment",
    "description": "Description",
    "status": "Status", 
    "lab": "Lab", 
    "document_len": "Doc", 
    #"construction",
    "has_processed": "Done",
    "could_process": "Ready",
    "organism": "Organism",
    #"construction_method": "Const. Method",
    "subcellular_fraction": "Cellular",
    #"biosample_ontology": "biosample ontology",
    "biosample_term": "bioterm",
    "sc_type": "sc_type",
}))
for row in sorted(results, key=lambda x: (x["status"], x["lab"], x["experiment"])):
    #print(report_format.format(row[0], row[1], row[2], len(row[3]), ",".join(row[4])[:20], ",".join(row[5])))
    print(report_format.format(**row))

Experiment   Status       Lab        Ready Done  Organism Cellular sc_type              bioterm
ENCSR005TCL  in progress  Stanford   Y     Y     human    nucleus  Chromium Multiome AT psoas muscle
ENCSR011EDH  in progress  Stanford   Y     Y     human    nucleus  Chromium Multiome AT fallopian tube
ENCSR012APQ  in progress  Stanford   Y     Y     human    nucleus  Chromium Multiome AT heart left ventricle
ENCSR021QIA  in progress  Stanford   Y     Y     human    nucleus  Chromium Multiome AT placenta
ENCSR021YYL  in progress  Stanford   Y     Y     mouse    nucleus  Chromium Multiome AT adrenal gland
ENCSR028MBY  in progress  Stanford   Y     Y     human    nucleus  Chromium GEM v3.1    mucosa of descending colon
ENCSR035BDI  in progress  Stanford   Y     Y     human    nucleus  Chromium GEM v3.1    adrenal gland
ENCSR060SEA  in progress  Stanford   Y     Y     human    nucleus  Chromium Multiome AT bile duct
ENCSR060ZXZ  in progress  Stanford   Y     Y     human    nucleus  Chromium M

In [10]:
pandas.DataFrame(results).sort_values(["status", "lab", "experiment"]).to_excel("scrna-experiments.xlsx", index=False)

In [11]:
print("Unprocessed {} of {}".format(sum([not x["has_processed"] for x in results]), len(results)))

Unprocessed 0 of 303


In [12]:
print("Unprocessed {} of {}".format(sum([(not x["has_processed"] and x["sc_type"].startswith("Chromium")) for x in results]), len(results)))

Unprocessed 0 of 303


In [13]:
print("Unprocessed {} of {}".format(sum([(x["could_process"] and x["sc_type"].startswith("Chromium") and x["lab"].startswith("Stanford"))  for x in results]), len(results)))

Unprocessed 107 of 303


In [14]:
print("Unprocessed {} of {}".format(sum([(not x["has_processed"] and x["sc_type"].startswith("Split-seq")) for x in results]), len(results)))

Unprocessed 0 of 303


In [15]:
# First I figured out what documents were present, then I made my mapping then 
# I produced the above report, but I lost generating what documents existed.
if 0:
    documents_seen = Counter()
    results = []
    needed_metadata = []
    for row in query["@graph"]:
        if row["status"] in ("in progress", "released"):

            experiment = server.get_json(row["@id"])
            documents = set(experiment.get("documents", []))
            construction_method = set()
            for replicate in experiment['replicates']:
                library = replicate['library']
                library_barcode = library.get("barcode_details")
                for d in library.get("documents", []):
                    documents.add(d)
                construction_platform = library.get('construction_platform', None)
                if construction_platform is not None:                
                    construction_method.add(construction_platform["term_name"])
                construction_platform = library.get("construction_method", None)
                if construction_platform is not None:
                    construction_method.add(construction_platform[0])

                biosample = library["biosample"]
                for d in biosample.get("documents", []):
                    documents.add(d)
                subcellular_fraction = biosample.get("subcellular_fraction_term_name")

            has_processed = False
            for f in experiment["files"]:
                if f["file_type"] == "fastq":
                    submitted_file_name = Path(f["submitted_file_name"])
                    print(experiment["accession"], f["accession"], f.get("paired_end"), f.get("paired_with"), submitted_file_name.name)
                    #for detail in f.get("flowcell_details", []):                    
                        #for barcode_attribute in ["barcode_in_read", "barcode", "barcode_position"]:
                        #    value = detail.get(barcode_attribute)
                        #    if value is not None:
                        #       print(row['accession'], barcode_attribute, value)
                else:
                    has_processed = True

            sc_type = set()
            for d in documents:
                assert d in known_document_types
                if d in document_to_protocol:
                    sc_type.add(document_to_protocol[d])
            documents_seen[",".join(experiment["documents"])] += 1

            results.append({
                "accession": row["accession"], 
                "status": row["status"], 
                "lab": row["lab"]["title"].split(',')[1], 
                "description": experiment.get("description", ""),
                "documents": documents,
                "document_len": len(documents),
                "construction_method": ",".join(construction_method),
                "library_barcode": library_barcode, 
                "sc_type": ",".join(sc_type),
                "subcellular_fraction": subcellular_fraction,
                "has_processed": has_processed,
            })

In [16]:
server.server

'www.encodeproject.org'

In [17]:
Filereps = namedtuple("Filereps", ["rep", "lane", "paired", "accession"])
illumina_parts = re.compile("_L(?P<lane>[\d]{3})_R(?P<read>[\d])_(?P<chunk>[\d]{3}).fastq.gz")

experiment = server.get_json("ENCSR724KET")

def generate_config(sc_type, experiment):
    paired_indexing = {
        "1": "indexed reads",
        "2": "reads",
    }
    
    accession = experiment["accession"]
    description = experiment.get("description", "")
    default_config = {
        "experiment_accession": accession,
        "description": description,
        "lab": "/labs/barbara-wold/",
        "award": "UM1HG009443",
        #"encode_portal_host": "encd-6248-78c147870-qing.demo.encodedcc.org",
        #"encode_portal_host": "encd-6248-24bd77146-qing.demo.encodedcc.org",
        "encode_portal_host": server.server,
        #"star_container": "https://woldlab.caltech.edu/~diane/containers/star-bash-dev_EoI-head.sif",
        "star_container": "https://woldlab.caltech.edu/~diane/containers/star-bash-2.7.10a.sif",
    
        # these should really scale some
        "mem_mb": 65536,
        "disk_mb": 51200,

        "automatic_submission": True
    }
    if sc_type not in protocol_defaults:
        print("Can't process {} {}: {}".format(accession, description, sc_type))
        return
    default_config.update(protocol_defaults[sc_type])

    fractions = set()
    config = {}
    for rep in experiment["replicates"]:
        library_id = rep["library"]["@id"]
        config.setdefault(library_id, {}).update(default_config)
        config[library_id]["library_accession"] = rep["library"]["accession"]
        biosample = rep["library"]["biosample"]
        organism_id = biosample['organism']['@id']
        if organism_id in default_genome_index:
            config[library_id].update(default_genome_index[organism_id])
        else:
            print("Unable to process {} {} until {} index".format(accession, description, organism_id))
        subcellular_fraction = biosample.get("subcellular_fraction_term_name", "")
        fractions.add(subcellular_fraction)
        if subcellular_fraction == "nucleus":
            config[library_id]["include_intron"] = True
        else:
            config[library_id]["include_intron"] = False

    assert(len(fractions)) == 1
    files = []
    for f in experiment["files"]:
        if f['file_format'] == 'fastq':
            submitted_file_name = Path(f["submitted_file_name"])
            lanes = []
            for detail in f.get("flowcell_details", []):
                if "lane" in detail and len(detail["lane"]) > 0:
                    lanes.append(detail["lane"])
            if len(lanes) == 0:
                match = illumina_parts.search(submitted_file_name.name)
                if match:
                    lanes.append(str(int(match.group("lane"))))
            lanes = ",".join(lanes)
            if len(lanes) == 0:
                print("Problems with", accession, library_id)
                
            file_library_id = f['replicate']["library"]
            if "paired_end" in f:
                read = paired_indexing[f["paired_end"]]
            elif f.get("output_type") in ('index reads', 'reads'):
                read = f["output_type"]
            else:
                raise ValueError("Unable to figure out which is the index read")
            
            fr = Filereps(file_library_id, lanes, read, f['accession'])
            #print(fr)
            #print(submitted_file_name)
            files.append(fr)

    files = sorted(files)
    for i, (read1, read2) in enumerate(zip(files[::2], files[1::2])):
        config[read1.rep].setdefault("read1", []).append(read1.accession)
        config[read1.rep].setdefault("read2", []).append(read2.accession)

    for library_id in config:
        yield config[library_id]


In [18]:
list(generate_config("Chromium Multiome ATAC + Gene Expression", experiment_cache["ENCSR005TCL"]))

[{'experiment_accession': 'ENCSR005TCL',
  'description': 'snRNA on human psoas muscle. Please note that this experiment is part of 10x multiome and has a correspoding scATAC experiment (ENCSR332XEW).',
  'lab': '/labs/barbara-wold/',
  'award': 'UM1HG009443',
  'encode_portal_host': 'www.encodeproject.org',
  'star_container': 'https://woldlab.caltech.edu/~diane/containers/star-bash-2.7.10a.sif',
  'mem_mb': 65536,
  'disk_mb': 51200,
  'automatic_submission': True,
  'stranded': 'Forward',
  'umi_length': 12,
  'inclusion_accession': '737K-arc-v1(GEX)',
  'library_accession': 'ENCLB549KAN',
  'genome_accession': 'encode:starsolo-GRCh38-V29-male-index',
  'include_intron': True,
  'read1': ['ENCFF121UIQ', 'ENCFF663ZRK'],
  'read2': ['ENCFF194KTM', 'ENCFF364QFJ']}]

In [19]:
list(generate_config("Chromium Multiome ATAC + Gene Expression", experiment_cache["ENCSR337HKS"]))

Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/


[{'experiment_accession': 'ENCSR337HKS',
  'description': '',
  'lab': '/labs/barbara-wold/',
  'award': 'UM1HG009443',
  'encode_portal_host': 'www.encodeproject.org',
  'star_container': 'https://woldlab.caltech.edu/~diane/containers/star-bash-2.7.10a.sif',
  'mem_mb': 65536,
  'disk_mb': 51200,
  'automatic_submission': True,
  'stranded': 'Forward',
  'umi_length': 12,
  'inclusion_accession': '737K-arc-v1(GEX)',
  'library_accession': 'ENCLB831IHU',
  'genome_accession': 'encode:starsolo-GRCh38-V29-male-index',
  'include_intron': True,
  'read1': ['ENCFF298GMK', 'ENCFF323AMX'],
  'read2': ['ENCFF931UDZ', 'ENCFF878VQP']}]

In [20]:
 list(generate_config("Split-seq", experiment_cache['ENCSR003FZY']))

Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/


[{'experiment_accession': 'ENCSR003FZY',
  'description': 'Short-read Split-seq B6Cast F1 P18-20mo male hippocampus deep',
  'lab': '/labs/barbara-wold/',
  'award': 'UM1HG009443',
  'encode_portal_host': 'www.encodeproject.org',
  'star_container': 'https://woldlab.caltech.edu/~diane/containers/star-bash-2.7.10a.sif',
  'mem_mb': 65536,
  'disk_mb': 51200,
  'automatic_submission': True,
  'stranded': 'Forward',
  'inclusion_list_url': 'https://woldlab.caltech.edu/~diane/genome/parse-splitseq-v1.tar.gz',
  'library_accession': 'ENCLB946ISI',
  'genome_accession': 'encode:starsolo-mm10-M21-male-index',
  'include_intron': True,
  'read1': ['ENCFF516EGU'],
  'read2': ['ENCFF101CWQ']},
 {'experiment_accession': 'ENCSR003FZY',
  'description': 'Short-read Split-seq B6Cast F1 P18-20mo male hippocampus deep',
  'lab': '/labs/barbara-wold/',
  'award': 'UM1HG009443',
  'encode_portal_host': 'www.encodeproject.org',
  'star_container': 'https://woldlab.caltech.edu/~diane/containers/star-bash-

In [21]:
all_configs = {}
for row in sorted(results, key=lambda x: (x["status"], x["lab"], x["experiment"])):
    if not row["has_processed"]:
        for config in generate_config(row["sc_type"], experiment_cache[row['experiment']]):
            library = config['library_accession']
            if config is not None:
                all_configs[library] = config

In [22]:
import yaml

In [23]:
print(yaml.dump(all_configs["ENCLB002DZK"]))

KeyError: 'ENCLB002DZK'

In [ ]:
print(yaml.dump(all_configs["ENCLB549KAN"]))

Ali said there's one multiome sample he'd like it processed

In [ ]:
print(yaml.dump(all_configs["ENCLB297FDO"]))

In [ ]:
print(yaml.dump(all_configs["ENCLB938HOJ"]))

In [ ]:
print(yaml.dump(all_configs["ENCLB527WWJ"]))

In [ ]:
print(yaml.dump(all_configs["ENCLB062TYC"]))

In [ ]:
print(yaml.dump(all_configs["ENCLB349KYC"]))

In [ ]:
print(yaml.dump(all_configs["ENCLB497BTJ"]))

In [ ]:
print(yaml.dump(all_configs["ENCLB549SMV"]))

In [ ]:
print("# splitseq")
print(yaml.dump(all_configs["ENCLB946ISI"]))

In [ ]:
print("# splitseq")
print(yaml.dump(all_configs["ENCLB194KKC"]))

In [ ]:
print("# splitseq")
print(yaml.dump(all_configs["ENCLB129LMS"]))

In [ ]:
print("# splitseq")
print(yaml.dump(all_configs["ENCLB952MZJ"]))

In [ ]:
print("# splitseq")
print(yaml.dump(all_configs["ENCLB098ERF"]))

In [ ]:
print("# splitseq")
print(yaml.dump(all_configs["ENCLB082IOS"]))

In [ ]:
def get_gene_model(config):
    return "GeneFull_Ex50pAS" if config['include_intron'] else "Gene"


def prepare_bulk_subworkflow(all_configs, libraries, submit=False):
    batch_config = []
    bulk_rule_input = []
    
    for library in libraries:
        config = all_configs[library]
        #config["automatic_submission"] = False
        submit_host = config["encode_portal_host"]
        library_dir = Path(library)
        workflow_name = "job_{}".format(library)

        current = """subworkflow {workflow_name}:
    workdir:
        "{workdir}"
    snakefile:
        "{snakefile}"
    configfile:
        "{configfile}"
""".format(
        workflow_name=workflow_name,
        workdir=str(library_dir),
        snakefile=str(Path("/home/diane/woldlab-rna-seq/workflow/process-encode-10x.snakefile")),
        configfile=str(library_dir / "config.yaml"))
        batch_config.append(current)
        
        posted_csv = "posted.{}.csv".format(submit_host)
        metadata_csv = "metadata.{}.csv".format(submit_host)
        target_rule = posted_csv if config["automatic_submission"] else metadata_csv
        bulk_rule_input.append("{}(\"{}\")".format(workflow_name, target_rule))
        final_out_rule = "Log.final.out.{}.qc-upload".format(submit_host)
        bulk_rule_input.append("{}(\"{}\")".format(workflow_name, final_out_rule))
        summary_rule = "Solo.out/{}/Summary.csv.{}.qc-upload".format(get_gene_model(config), submit_host)
        bulk_rule_input.append("{}(\"{}\")".format(workflow_name, summary_rule))
        pct_count_mt_rule = "pct_count_mt.{}_EM_filtered.png.{}.qc-upload".format(get_gene_model(config), submit_host)
        bulk_rule_input.append("{}(\"{}\")".format(workflow_name, pct_count_mt_rule))
    
    formatted_rules = []
    for rule in bulk_rule_input:
        formatted_rules.append("{prefix}{rule},\n".format(prefix=" " * 8, rule=rule))
    batch_config.append("""rule world:
    input:
{rules}
""".format(rules="".join(formatted_rules)))
    return "\n".join(batch_config)
        

In [ ]:
print(prepare_bulk_subworkflow(all_configs, ["ENCLB002DZK", "ENCLB549KAN"]))

In [ ]:
def prepare_bulk_subworkflow_run(destination, all_configs, libraries):
    with tarfile.open(destination, "w:gz") as archive:
        for library in libraries:
            library_dir = Path(library)
            config_data = yaml.dump(all_configs[library]).encode("utf-8")
            config_stream = BytesIO(config_data)
            config_filename = library_dir / "config.yaml"
            config_info = tarfile.TarInfo(str(config_filename))
            config_info.size = len(config_data)
            
            archive.addfile(config_info, fileobj=config_stream)
            
        world_data = prepare_bulk_subworkflow(all_configs, libraries).encode("utf-8")
        world_stream = BytesIO(world_data)
        world_info = tarfile.TarInfo("Snakefile")
        world_info.size = len(world_data)
        archive.addfile(world_info, world_stream)

In [ ]:
prepare_bulk_subworkflow_run("/dev/shm/all.tar.gz", all_configs, ["ENCLB002DZK", "ENCLB549KAN"])

In [ ]:
def prepare_bulk_sbatch(all_configs, libraries, submit=False):
    commands = ["#!/bin/bash", ""]
    
    for library in libraries:
        config = all_configs[library]
        library_dir = Path(library)

        commands.append("pushd {}".format(library_dir))
        commands.append("sbatch --time 1-0 --mem 64GB --cpus-per-task 24 -- snakemake --use-singularity --snakefile ~/woldlab-rna-seq/workflow/process-encode-10x.snakefile --cluster \"sbatch --time 1-0 --mem {resources.mem_mb} --cpus-per-task {threads}\" --jobs 40 --cores 40")
        commands.append("popd")
    
    return "\n".join(commands)
        

In [ ]:
def prepare_bulk_sbatch_run(destination, all_configs, libraries):
    with tarfile.open(destination, "w:gz") as archive:
        for library in libraries:
            library_dir = Path(library)
            config_data = yaml.dump(all_configs[library]).encode("utf-8")
            config_stream = BytesIO(config_data)
            config_filename = library_dir / "config.yaml"
            config_info = tarfile.TarInfo(str(config_filename))
            config_info.size = len(config_data)
            
            archive.addfile(config_info, fileobj=config_stream)
            
        world_data = prepare_bulk_sbatch(all_configs, libraries).encode("utf-8")
        world_stream = BytesIO(world_data)
        world_info = tarfile.TarInfo("run.sh")
        world_info.size = len(world_data)
        archive.addfile(world_info, world_stream)

In [ ]:
metadata = pandas.DataFrame(results)

In [ ]:
experiments_to_run_2_6 = ["ENCSR005TCL","ENCSR011EDH","ENCSR012APQ","ENCSR021QIA","ENCSR021YYL"]

experiments_to_run_7_21 = [
"ENCSR028MBY",
"ENCSR035BDI",
"ENCSR060SEA",
"ENCSR060ZXZ",
"ENCSR075HVN",
"ENCSR079LVG",
"ENCSR087YKG",
"ENCSR132MEW",
"ENCSR133XJX",
"ENCSR142RRV",
"ENCSR151ONW",
"ENCSR157YJO",
"ENCSR201WJO",
"ENCSR203THX",
"ENCSR224OUG",
]

#experiments_to_run = set(["ENCSR402PHR","ENCSR157YJO","ENCSR021YYL","ENCSR988AMY","ENCSR908CQZ","ENCSR749GDE","ENCSR224OUG","ENCSR356VJZ", "ENCSR021YYL"])
experiments_to_run_22_51 = [
"ENCSR224RWH",
"ENCSR225OII",
"ENCSR258SZI",
"ENCSR259VOY",
"ENCSR261BXB",
"ENCSR261LPM",
"ENCSR281NBH",
"ENCSR281RIH",
"ENCSR289ANH",
"ENCSR297ZTQ",
"ENCSR298MPV",
"ENCSR333SLJ",
"ENCSR337HKS",
"ENCSR352DXB",
"ENCSR356VJZ",
"ENCSR362HUP",
"ENCSR362YDM",
"ENCSR367YUX",
"ENCSR369UFT",
"ENCSR398PIW",
"ENCSR402PHR",
"ENCSR405YKM",
"ENCSR415HEP",
"ENCSR417FCM",
"ENCSR421YXX",
"ENCSR426PDU",
"ENCSR439ZVQ",
"ENCSR445LCA",
"ENCSR460MGC",
"ENCSR462HCD",
]

experiments_to_run_52_81 = [
"ENCSR463EEP",
"ENCSR467GGT",
"ENCSR471YTA",
"ENCSR472RRP",
"ENCSR478FQR",
"ENCSR481QQR",
"ENCSR484YSK",
"ENCSR522HNR",
"ENCSR527MGL",
"ENCSR533IGQ",
"ENCSR539XHZ",
"ENCSR548HED",
"ENCSR568GGW",
"ENCSR587SPO",
"ENCSR602SGU",
"ENCSR620AJZ",
"ENCSR624WDN",
"ENCSR632EIK",
"ENCSR641MWO",
"ENCSR642GFM",
"ENCSR644VVE",
"ENCSR656VWP",
"ENCSR662VTU",
"ENCSR665HPR",
"ENCSR671FDY",
"ENCSR683ONV",
"ENCSR684KYI",
"ENCSR700UVN",
"ENCSR707ECV",
"ENCSR707LLI",
]

experiments_to_run_82_107 = ["ENCSR716VLR",
"ENCSR720TUP",
"ENCSR724KET",
"ENCSR726IPC",
"ENCSR727OYO",
"ENCSR742NWO",
"ENCSR749GDE",
"ENCSR751BHQ",
"ENCSR753YPW",
"ENCSR755CIF",
"ENCSR762LZV",
"ENCSR769KWM",
"ENCSR789PLK",
"ENCSR801DHT",
"ENCSR813PWQ",
"ENCSR835JFN",
"ENCSR876EUH",
"ENCSR908CQZ",
"ENCSR938CZJ",
"ENCSR943HBF",
"ENCSR944XPC",
"ENCSR964JZN",
"ENCSR966DDY",
"ENCSR968RPR",
"ENCSR987SOK",
"ENCSR988AMY",
"ENCSR995ZKH",
]

experiments_to_run = experiments_to_run_82_107
print("Preparing {} jobs".format(len(experiments_to_run)))
ready_metadata = metadata[metadata["experiment"].isin(experiments_to_run)]

#prepare_bulk_sbatch_run("/dev/shm/test-run-82-108.tar.gz", all_configs, ready_metadata["library"].values)

In [ ]:
#prepare_bulk_run("/dev/shm/adrenal_multiome.tar.gz", all_configs, ready_metadata["library"].values)


In [ ]:
for i, row in ready_metadata.iterrows():
    config = all_configs[row.library]
    print("https://{}/experiments/{}/".format(config["encode_portal_host"], row.experiment))

In [ ]:
prepare_bulk_sbatch_run("/dev/shm/run-1-108.tar.gz", all_configs, metadata[metadata["lab"] == "Stanford"]["library"].values)

In [ ]:
prepare_bulk_sbatch_run("/dev/shm/test-run-82-108.tar.gz", all_configs, ready_metadata["library"].values)

In [ ]:
def get_dcc_accession(server, accession):
    metadata = None
    try:
        metadata = server.get_json(accession)
    except HTTPError as e:
        if e.response.status_code == 404:
            print("Accession {} not found, discovery will fail".format(accession))
        else:
            print("Other error {} for {}".format(e.response.status_code, accession))
            raise e

    return metadata



In [ ]:
from io import StringIO
submitted = pandas.read_csv(StringIO("""uuid,accession,dataset,file_format,output_type,assembly,genome_annotation,derived_from:array,md5sum,file_size:integer,submitted_file_name,award,lab,step_run,aliases:array
,,ENCSR224OUG,bam,alignments,mm10,M21,"encode:starsolo-mm10-M21-male-index,/files/ENCFF494DXJ/,/files/ENCFF586ZOY/,/files/ENCFF201AFX/,/files/ENCFF468KNY/",38bf053481a36b565a45e2e4fedb2bbf,14365291492,Aligned.sortedByCoord.out.bam,UM1HG009443,/labs/barbara-wold/,barbara-wold:starsolo-10x-alignment-step-run,barbara-wold:ENCLB607FDJ_alignment_2022-01-19
,,ENCSR224OUG,tar,sparse gene count matrix of unique reads,mm10,M21,"737K-arc-v1(GEX),barbara-wold:ENCLB607FDJ_alignment_2022-01-19",d4214b016b19730e185ce11dd5396514,39391130,GeneFull_Ex50pAS_Unique_filtered.tar.gz,UM1HG009443,/labs/barbara-wold/,barbara-wold:starsolo-10x-quantification-step-run,
,,ENCSR224OUG,tar,sparse gene count matrix of all reads,mm10,M21,"737K-arc-v1(GEX),barbara-wold:ENCLB607FDJ_alignment_2022-01-19",68d9aa21aae8e891f85f880cd5bd8cab,46821115,GeneFull_Ex50pAS_EM_filtered.tar.gz,UM1HG009443,/labs/barbara-wold/,barbara-wold:starsolo-10x-quantification-step-run,
,,ENCSR224OUG,tar,unfiltered sparse gene count matrix of unique reads,mm10,M21,"737K-arc-v1(GEX),barbara-wold:ENCLB607FDJ_alignment_2022-01-19",920b78efae954ffb082faf2aaa921ba5,59389166,GeneFull_Ex50pAS_Unique_raw.tar.gz,UM1HG009443,/labs/barbara-wold/,barbara-wold:starsolo-10x-quantification-step-run,
,,ENCSR224OUG,tar,unfiltered sparse gene count matrix of all reads,mm10,M21,"737K-arc-v1(GEX),barbara-wold:ENCLB607FDJ_alignment_2022-01-19",7d6019e2816f28f19eeecd47a3442608,76437934,GeneFull_Ex50pAS_EM_raw.tar.gz,UM1HG009443,/labs/barbara-wold/,barbara-wold:starsolo-10x-quantification-step-run,
,,ENCSR224OUG,tar,unfiltered sparse splice junction count matrix of unique reads,mm10,M21,"737K-arc-v1(GEX),barbara-wold:ENCLB607FDJ_alignment_2022-01-19",2dd5935279637f61abf89e5778881a77,14019963,SJ_Unique_raw.tar.gz,UM1HG009443,/labs/barbara-wold/,barbara-wold:starsolo-10x-quantification-step-run,
"""))
submitted

In [ ]:
problems = set("""Problems with ENCSR203THX /libraries/ENCLB592LXS/
Problems with ENCSR203THX /libraries/ENCLB592LXS/
Problems with ENCSR203THX /libraries/ENCLB592LXS/
Problems with ENCSR203THX /libraries/ENCLB592LXS/
Problems with ENCSR298MPV /libraries/ENCLB814OYP/
Problems with ENCSR298MPV /libraries/ENCLB814OYP/
Problems with ENCSR298MPV /libraries/ENCLB814OYP/
Problems with ENCSR298MPV /libraries/ENCLB814OYP/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR337HKS /libraries/ENCLB831IHU/
Problems with ENCSR405YKM /libraries/ENCLB181TPV/
Problems with ENCSR405YKM /libraries/ENCLB181TPV/
Problems with ENCSR405YKM /libraries/ENCLB181TPV/
Problems with ENCSR405YKM /libraries/ENCLB181TPV/
Problems with ENCSR463EEP /libraries/ENCLB619VVF/
Problems with ENCSR463EEP /libraries/ENCLB619VVF/
Problems with ENCSR463EEP /libraries/ENCLB619VVF/
Problems with ENCSR463EEP /libraries/ENCLB619VVF/
Problems with ENCSR527MGL /libraries/ENCLB909DQG/
Problems with ENCSR527MGL /libraries/ENCLB909DQG/
Problems with ENCSR527MGL /libraries/ENCLB909DQG/
Problems with ENCSR527MGL /libraries/ENCLB909DQG/
Problems with ENCSR662VTU /libraries/ENCLB615BOP/
Problems with ENCSR662VTU /libraries/ENCLB615BOP/
Problems with ENCSR662VTU /libraries/ENCLB615BOP/
Problems with ENCSR662VTU /libraries/ENCLB615BOP/
Problems with ENCSR700UVN /libraries/ENCLB527WWJ/
Problems with ENCSR700UVN /libraries/ENCLB527WWJ/
Problems with ENCSR700UVN /libraries/ENCLB527WWJ/
Problems with ENCSR700UVN /libraries/ENCLB527WWJ/
Problems with ENCSR769KWM /libraries/ENCLB812SMT/
Problems with ENCSR769KWM /libraries/ENCLB812SMT/
Problems with ENCSR769KWM /libraries/ENCLB812SMT/
Problems with ENCSR769KWM /libraries/ENCLB812SMT/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR003FZY /libraries/ENCLB194KKC/
Problems with ENCSR024CHH /libraries/ENCLB491BDA/
Problems with ENCSR024CHH /libraries/ENCLB491BDA/
Problems with ENCSR029GVP /libraries/ENCLB631JOU/
Problems with ENCSR029GVP /libraries/ENCLB631JOU/
Problems with ENCSR029GVP /libraries/ENCLB631JOU/
Problems with ENCSR029GVP /libraries/ENCLB631JOU/
Problems with ENCSR029OIF /libraries/ENCLB048POP/
Problems with ENCSR029OIF /libraries/ENCLB048POP/
Problems with ENCSR029OIF /libraries/ENCLB048POP/
Problems with ENCSR029OIF /libraries/ENCLB048POP/
Problems with ENCSR035SWS /libraries/ENCLB315XSJ/
Problems with ENCSR035SWS /libraries/ENCLB315XSJ/
Problems with ENCSR035SWS /libraries/ENCLB315XSJ/
Problems with ENCSR035SWS /libraries/ENCLB315XSJ/
Problems with ENCSR043HKT /libraries/ENCLB771NKO/
Problems with ENCSR043HKT /libraries/ENCLB771NKO/
Problems with ENCSR043HKT /libraries/ENCLB771NKO/
Problems with ENCSR043HKT /libraries/ENCLB771NKO/
Problems with ENCSR044GSN /libraries/ENCLB102VRS/
Problems with ENCSR044GSN /libraries/ENCLB102VRS/
Problems with ENCSR044GSN /libraries/ENCLB102VRS/
Problems with ENCSR044GSN /libraries/ENCLB102VRS/
Problems with ENCSR062ICV /libraries/ENCLB440ABL/
Problems with ENCSR062ICV /libraries/ENCLB440ABL/
Problems with ENCSR062ICV /libraries/ENCLB440ABL/
Problems with ENCSR062ICV /libraries/ENCLB440ABL/
Problems with ENCSR062WSS /libraries/ENCLB046ZBE/
Problems with ENCSR062WSS /libraries/ENCLB046ZBE/
Problems with ENCSR062WSS /libraries/ENCLB046ZBE/
Problems with ENCSR062WSS /libraries/ENCLB046ZBE/
Problems with ENCSR080VLK /libraries/ENCLB400DWX/
Problems with ENCSR080VLK /libraries/ENCLB400DWX/
Problems with ENCSR081SRK /libraries/ENCLB798BJP/
Problems with ENCSR081SRK /libraries/ENCLB798BJP/
Problems with ENCSR081SRK /libraries/ENCLB798BJP/
Problems with ENCSR081SRK /libraries/ENCLB798BJP/
Problems with ENCSR087LWL /libraries/ENCLB892ZAW/
Problems with ENCSR087LWL /libraries/ENCLB892ZAW/
Problems with ENCSR087LWL /libraries/ENCLB892ZAW/
Problems with ENCSR087LWL /libraries/ENCLB892ZAW/
Problems with ENCSR090OFY /libraries/ENCLB061KPS/
Problems with ENCSR090OFY /libraries/ENCLB061KPS/
Problems with ENCSR090OFY /libraries/ENCLB061KPS/
Problems with ENCSR090OFY /libraries/ENCLB061KPS/
Problems with ENCSR093ZTB /libraries/ENCLB988YCJ/
Problems with ENCSR093ZTB /libraries/ENCLB988YCJ/
Problems with ENCSR093ZTB /libraries/ENCLB988YCJ/
Problems with ENCSR093ZTB /libraries/ENCLB988YCJ/
Problems with ENCSR096PUV /libraries/ENCLB169FGN/
Problems with ENCSR096PUV /libraries/ENCLB169FGN/
Problems with ENCSR096PUV /libraries/ENCLB169FGN/
Problems with ENCSR096PUV /libraries/ENCLB169FGN/
Problems with ENCSR105JHX /libraries/ENCLB792DPM/
Problems with ENCSR105JHX /libraries/ENCLB792DPM/
Problems with ENCSR105JHX /libraries/ENCLB792DPM/
Problems with ENCSR105JHX /libraries/ENCLB792DPM/
Problems with ENCSR115JCE /libraries/ENCLB156LUM/
Problems with ENCSR115JCE /libraries/ENCLB156LUM/
Problems with ENCSR115JCE /libraries/ENCLB156LUM/
Problems with ENCSR115JCE /libraries/ENCLB156LUM/
Problems with ENCSR119HVP /libraries/ENCLB372GFV/
Problems with ENCSR119HVP /libraries/ENCLB372GFV/
Problems with ENCSR119HVP /libraries/ENCLB372GFV/
Problems with ENCSR119HVP /libraries/ENCLB372GFV/
Problems with ENCSR120UHU /libraries/ENCLB343QQX/
Problems with ENCSR120UHU /libraries/ENCLB343QQX/
Problems with ENCSR120UHU /libraries/ENCLB343QQX/
Problems with ENCSR120UHU /libraries/ENCLB343QQX/
Problems with ENCSR126YAO /libraries/ENCLB848YMF/
Problems with ENCSR126YAO /libraries/ENCLB848YMF/
Problems with ENCSR126YAO /libraries/ENCLB848YMF/
Problems with ENCSR126YAO /libraries/ENCLB848YMF/
Problems with ENCSR135ELG /libraries/ENCLB722NFY/
Problems with ENCSR135ELG /libraries/ENCLB722NFY/
Problems with ENCSR145DBY /libraries/ENCLB230VGI/
Problems with ENCSR145DBY /libraries/ENCLB230VGI/
Problems with ENCSR146AFJ /libraries/ENCLB323HIM/
Problems with ENCSR146AFJ /libraries/ENCLB323HIM/
Problems with ENCSR146AFJ /libraries/ENCLB323HIM/
Problems with ENCSR146AFJ /libraries/ENCLB323HIM/
Problems with ENCSR147UYS /libraries/ENCLB564RVH/
Problems with ENCSR147UYS /libraries/ENCLB564RVH/
Problems with ENCSR147UYS /libraries/ENCLB564RVH/
Problems with ENCSR147UYS /libraries/ENCLB564RVH/
Problems with ENCSR156BQX /libraries/ENCLB894TFF/
Problems with ENCSR156BQX /libraries/ENCLB894TFF/
Problems with ENCSR156BQX /libraries/ENCLB894TFF/
Problems with ENCSR156BQX /libraries/ENCLB894TFF/
Problems with ENCSR162LWP /libraries/ENCLB672PSR/
Problems with ENCSR162LWP /libraries/ENCLB672PSR/
Problems with ENCSR162LWP /libraries/ENCLB672PSR/
Problems with ENCSR162LWP /libraries/ENCLB672PSR/
Problems with ENCSR164REI /libraries/ENCLB249TPV/
Problems with ENCSR164REI /libraries/ENCLB249TPV/
Problems with ENCSR164REI /libraries/ENCLB249TPV/
Problems with ENCSR164REI /libraries/ENCLB249TPV/
Problems with ENCSR165CAC /libraries/ENCLB323CCI/
Problems with ENCSR165CAC /libraries/ENCLB323CCI/
Problems with ENCSR169QXA /libraries/ENCLB213LKC/
Problems with ENCSR169QXA /libraries/ENCLB213LKC/
Problems with ENCSR169QXA /libraries/ENCLB213LKC/
Problems with ENCSR169QXA /libraries/ENCLB213LKC/
Problems with ENCSR171BEG /libraries/ENCLB143IPZ/
Problems with ENCSR171BEG /libraries/ENCLB143IPZ/
Problems with ENCSR171BEG /libraries/ENCLB143IPZ/
Problems with ENCSR171BEG /libraries/ENCLB143IPZ/
Problems with ENCSR172XWR /libraries/ENCLB820VCW/
Problems with ENCSR172XWR /libraries/ENCLB820VCW/
Problems with ENCSR172XWR /libraries/ENCLB820VCW/
Problems with ENCSR172XWR /libraries/ENCLB820VCW/
Problems with ENCSR178WRO /libraries/ENCLB829GDB/
Problems with ENCSR178WRO /libraries/ENCLB829GDB/
Problems with ENCSR186OUD /libraries/ENCLB072RWF/
Problems with ENCSR186OUD /libraries/ENCLB072RWF/
Problems with ENCSR186OUD /libraries/ENCLB072RWF/
Problems with ENCSR186OUD /libraries/ENCLB072RWF/
Problems with ENCSR186TDH /libraries/ENCLB716MVI/
Problems with ENCSR186TDH /libraries/ENCLB716MVI/
Problems with ENCSR186TDH /libraries/ENCLB716MVI/
Problems with ENCSR186TDH /libraries/ENCLB716MVI/
Problems with ENCSR188BZZ /libraries/ENCLB603YRZ/
Problems with ENCSR188BZZ /libraries/ENCLB603YRZ/
Problems with ENCSR188BZZ /libraries/ENCLB603YRZ/
Problems with ENCSR188BZZ /libraries/ENCLB603YRZ/
Problems with ENCSR191YNY /libraries/ENCLB266ZRP/
Problems with ENCSR191YNY /libraries/ENCLB266ZRP/
Problems with ENCSR192TYH /libraries/ENCLB925LWZ/
Problems with ENCSR192TYH /libraries/ENCLB925LWZ/
Problems with ENCSR192TYH /libraries/ENCLB925LWZ/
Problems with ENCSR192TYH /libraries/ENCLB925LWZ/
Problems with ENCSR193PLQ /libraries/ENCLB082IOS/
Problems with ENCSR193PLQ /libraries/ENCLB082IOS/
Problems with ENCSR193PLQ /libraries/ENCLB082IOS/
Problems with ENCSR193PLQ /libraries/ENCLB082IOS/
Problems with ENCSR204FUI /libraries/ENCLB072YJA/
Problems with ENCSR204FUI /libraries/ENCLB072YJA/
Problems with ENCSR204FUI /libraries/ENCLB072YJA/
Problems with ENCSR204FUI /libraries/ENCLB072YJA/
Problems with ENCSR226SBC /libraries/ENCLB889CHC/
Problems with ENCSR226SBC /libraries/ENCLB889CHC/
Problems with ENCSR226SBC /libraries/ENCLB889CHC/
Problems with ENCSR226SBC /libraries/ENCLB889CHC/
Problems with ENCSR227CPT /libraries/ENCLB793OLP/
Problems with ENCSR227CPT /libraries/ENCLB793OLP/
Problems with ENCSR227CPT /libraries/ENCLB793OLP/
Problems with ENCSR227CPT /libraries/ENCLB793OLP/
Problems with ENCSR236MGX /libraries/ENCLB692PHT/
Problems with ENCSR236MGX /libraries/ENCLB692PHT/
Problems with ENCSR236MGX /libraries/ENCLB692PHT/
Problems with ENCSR236MGX /libraries/ENCLB692PHT/
Problems with ENCSR237EZS /libraries/ENCLB010VHV/
Problems with ENCSR237EZS /libraries/ENCLB010VHV/
Problems with ENCSR237EZS /libraries/ENCLB010VHV/
Problems with ENCSR237EZS /libraries/ENCLB010VHV/
Problems with ENCSR240CYU /libraries/ENCLB601XFB/
Problems with ENCSR240CYU /libraries/ENCLB601XFB/
Problems with ENCSR240CYU /libraries/ENCLB601XFB/
Problems with ENCSR240CYU /libraries/ENCLB601XFB/
Problems with ENCSR244FME /libraries/ENCLB590EJB/
Problems with ENCSR244FME /libraries/ENCLB590EJB/
Problems with ENCSR244FME /libraries/ENCLB590EJB/
Problems with ENCSR244FME /libraries/ENCLB590EJB/
Problems with ENCSR248IOH /libraries/ENCLB570NOI/
Problems with ENCSR248IOH /libraries/ENCLB570NOI/
Problems with ENCSR248IOH /libraries/ENCLB570NOI/
Problems with ENCSR248IOH /libraries/ENCLB570NOI/
Problems with ENCSR250ZVV /libraries/ENCLB274ZBT/
Problems with ENCSR250ZVV /libraries/ENCLB274ZBT/
Problems with ENCSR250ZVV /libraries/ENCLB274ZBT/
Problems with ENCSR250ZVV /libraries/ENCLB274ZBT/
Problems with ENCSR266HBP /libraries/ENCLB931EHJ/
Problems with ENCSR266HBP /libraries/ENCLB931EHJ/
Problems with ENCSR266HBP /libraries/ENCLB931EHJ/
Problems with ENCSR266HBP /libraries/ENCLB931EHJ/
Problems with ENCSR268TST /libraries/ENCLB548SZL/
Problems with ENCSR268TST /libraries/ENCLB548SZL/
Problems with ENCSR284OLV /libraries/ENCLB164ZHY/
Problems with ENCSR284OLV /libraries/ENCLB164ZHY/
Problems with ENCSR284OLV /libraries/ENCLB164ZHY/
Problems with ENCSR284OLV /libraries/ENCLB164ZHY/
Problems with ENCSR287VRB /libraries/ENCLB544SEN/
Problems with ENCSR287VRB /libraries/ENCLB544SEN/
Problems with ENCSR287VRB /libraries/ENCLB544SEN/
Problems with ENCSR287VRB /libraries/ENCLB544SEN/
Problems with ENCSR299DGH /libraries/ENCLB773PPF/
Problems with ENCSR299DGH /libraries/ENCLB773PPF/
Problems with ENCSR299DGH /libraries/ENCLB773PPF/
Problems with ENCSR299DGH /libraries/ENCLB773PPF/
Problems with ENCSR306ZET /libraries/ENCLB018ZYA/
Problems with ENCSR306ZET /libraries/ENCLB018ZYA/
Problems with ENCSR306ZET /libraries/ENCLB018ZYA/
Problems with ENCSR306ZET /libraries/ENCLB018ZYA/
Problems with ENCSR311TRI /libraries/ENCLB321YXH/
Problems with ENCSR311TRI /libraries/ENCLB321YXH/
Problems with ENCSR311TRI /libraries/ENCLB321YXH/
Problems with ENCSR311TRI /libraries/ENCLB321YXH/
Problems with ENCSR321BZA /libraries/ENCLB715WPG/
Problems with ENCSR321BZA /libraries/ENCLB715WPG/
Problems with ENCSR322KUB /libraries/ENCLB908NNT/
Problems with ENCSR322KUB /libraries/ENCLB908NNT/
Problems with ENCSR322KUB /libraries/ENCLB908NNT/
Problems with ENCSR322KUB /libraries/ENCLB908NNT/
Problems with ENCSR327PSD /libraries/ENCLB919KOR/
Problems with ENCSR327PSD /libraries/ENCLB919KOR/
Problems with ENCSR331DYV /libraries/ENCLB905KRE/
Problems with ENCSR331DYV /libraries/ENCLB905KRE/
Problems with ENCSR331DYV /libraries/ENCLB905KRE/
Problems with ENCSR331DYV /libraries/ENCLB905KRE/
Problems with ENCSR331KXG /libraries/ENCLB661DWA/
Problems with ENCSR331KXG /libraries/ENCLB661DWA/
Problems with ENCSR334YXW /libraries/ENCLB457VVB/
Problems with ENCSR334YXW /libraries/ENCLB457VVB/
Problems with ENCSR345BZU /libraries/ENCLB211BVF/
Problems with ENCSR345BZU /libraries/ENCLB211BVF/
Problems with ENCSR356NFK /libraries/ENCLB797XAC/
Problems with ENCSR356NFK /libraries/ENCLB797XAC/
Problems with ENCSR356NFK /libraries/ENCLB797XAC/
Problems with ENCSR356NFK /libraries/ENCLB797XAC/
Problems with ENCSR356PVK /libraries/ENCLB604HIX/
Problems with ENCSR356PVK /libraries/ENCLB604HIX/
Problems with ENCSR356PVK /libraries/ENCLB604HIX/
Problems with ENCSR356PVK /libraries/ENCLB604HIX/
Problems with ENCSR358XDG /libraries/ENCLB239BXF/
Problems with ENCSR358XDG /libraries/ENCLB239BXF/
Problems with ENCSR358XDG /libraries/ENCLB239BXF/
Problems with ENCSR358XDG /libraries/ENCLB239BXF/
Problems with ENCSR366MPW /libraries/ENCLB129SKD/
Problems with ENCSR366MPW /libraries/ENCLB129SKD/
Problems with ENCSR366MPW /libraries/ENCLB129SKD/
Problems with ENCSR366MPW /libraries/ENCLB129SKD/
Problems with ENCSR382ZGH /libraries/ENCLB988KOO/
Problems with ENCSR382ZGH /libraries/ENCLB988KOO/
Problems with ENCSR382ZGH /libraries/ENCLB988KOO/
Problems with ENCSR382ZGH /libraries/ENCLB988KOO/
Problems with ENCSR388RND /libraries/ENCLB943OZA/
Problems with ENCSR388RND /libraries/ENCLB943OZA/
Problems with ENCSR388RND /libraries/ENCLB943OZA/
Problems with ENCSR388RND /libraries/ENCLB943OZA/
Problems with ENCSR389TZP /libraries/ENCLB306NVG/
Problems with ENCSR389TZP /libraries/ENCLB306NVG/
Problems with ENCSR389TZP /libraries/ENCLB306NVG/
Problems with ENCSR389TZP /libraries/ENCLB306NVG/
Problems with ENCSR390GEG /libraries/ENCLB690LBQ/
Problems with ENCSR390GEG /libraries/ENCLB690LBQ/
Problems with ENCSR390GEG /libraries/ENCLB690LBQ/
Problems with ENCSR390GEG /libraries/ENCLB690LBQ/
Problems with ENCSR400PBH /libraries/ENCLB010FKE/
Problems with ENCSR400PBH /libraries/ENCLB010FKE/
Problems with ENCSR400PBH /libraries/ENCLB010FKE/
Problems with ENCSR400PBH /libraries/ENCLB010FKE/
Problems with ENCSR405CPX /libraries/ENCLB614YSB/
Problems with ENCSR405CPX /libraries/ENCLB614YSB/
Problems with ENCSR405CPX /libraries/ENCLB614YSB/
Problems with ENCSR405CPX /libraries/ENCLB614YSB/
Problems with ENCSR405RFP /libraries/ENCLB261MTS/
Problems with ENCSR405RFP /libraries/ENCLB261MTS/
Problems with ENCSR405RFP /libraries/ENCLB261MTS/
Problems with ENCSR405RFP /libraries/ENCLB261MTS/
Problems with ENCSR408PZM /libraries/ENCLB336ZNO/
Problems with ENCSR408PZM /libraries/ENCLB336ZNO/
Problems with ENCSR408PZM /libraries/ENCLB336ZNO/
Problems with ENCSR408PZM /libraries/ENCLB336ZNO/
Problems with ENCSR408ZUR /libraries/ENCLB382NBT/
Problems with ENCSR408ZUR /libraries/ENCLB382NBT/
Problems with ENCSR408ZUR /libraries/ENCLB382NBT/
Problems with ENCSR408ZUR /libraries/ENCLB382NBT/
Problems with ENCSR415CYS /libraries/ENCLB867BPN/
Problems with ENCSR415CYS /libraries/ENCLB867BPN/
Problems with ENCSR415CYS /libraries/ENCLB867BPN/
Problems with ENCSR415CYS /libraries/ENCLB867BPN/
Problems with ENCSR421VUT /libraries/ENCLB673SUG/
Problems with ENCSR421VUT /libraries/ENCLB673SUG/
Problems with ENCSR421VUT /libraries/ENCLB673SUG/
Problems with ENCSR421VUT /libraries/ENCLB673SUG/
Problems with ENCSR423AXI /libraries/ENCLB489WXX/
Problems with ENCSR423AXI /libraries/ENCLB489WXX/
Problems with ENCSR423AXI /libraries/ENCLB489WXX/
Problems with ENCSR423AXI /libraries/ENCLB489WXX/
Problems with ENCSR423CNV /libraries/ENCLB622RLX/
Problems with ENCSR423CNV /libraries/ENCLB622RLX/
Problems with ENCSR423CNV /libraries/ENCLB622RLX/
Problems with ENCSR423CNV /libraries/ENCLB622RLX/
Problems with ENCSR423LFS /libraries/ENCLB516LRK/
Problems with ENCSR423LFS /libraries/ENCLB516LRK/
Problems with ENCSR423LFS /libraries/ENCLB516LRK/
Problems with ENCSR423LFS /libraries/ENCLB516LRK/
Problems with ENCSR424RYS /libraries/ENCLB228MHF/
Problems with ENCSR424RYS /libraries/ENCLB228MHF/
Problems with ENCSR424RYS /libraries/ENCLB228MHF/
Problems with ENCSR424RYS /libraries/ENCLB228MHF/
Problems with ENCSR426MLQ /libraries/ENCLB368QMP/
Problems with ENCSR426MLQ /libraries/ENCLB368QMP/
Problems with ENCSR427OIY /libraries/ENCLB319YUS/
Problems with ENCSR427OIY /libraries/ENCLB319YUS/
Problems with ENCSR427OIY /libraries/ENCLB319YUS/
Problems with ENCSR427OIY /libraries/ENCLB319YUS/
Problems with ENCSR429ZKT /libraries/ENCLB523JWN/
Problems with ENCSR429ZKT /libraries/ENCLB523JWN/
Problems with ENCSR429ZKT /libraries/ENCLB523JWN/
Problems with ENCSR429ZKT /libraries/ENCLB523JWN/
Problems with ENCSR441YBL /libraries/ENCLB903BTS/
Problems with ENCSR441YBL /libraries/ENCLB903BTS/
Problems with ENCSR441YBL /libraries/ENCLB903BTS/
Problems with ENCSR441YBL /libraries/ENCLB903BTS/
Problems with ENCSR462YAG /libraries/ENCLB954TLZ/
Problems with ENCSR462YAG /libraries/ENCLB954TLZ/
Problems with ENCSR462YAG /libraries/ENCLB954TLZ/
Problems with ENCSR462YAG /libraries/ENCLB954TLZ/
Problems with ENCSR470GMO /libraries/ENCLB592ZMI/
Problems with ENCSR470GMO /libraries/ENCLB592ZMI/
Problems with ENCSR470GMO /libraries/ENCLB592ZMI/
Problems with ENCSR470GMO /libraries/ENCLB592ZMI/
Problems with ENCSR477EIC /libraries/ENCLB983QCI/
Problems with ENCSR477EIC /libraries/ENCLB983QCI/
Problems with ENCSR488RZS /libraries/ENCLB989CJP/
Problems with ENCSR488RZS /libraries/ENCLB989CJP/
Problems with ENCSR488RZS /libraries/ENCLB989CJP/
Problems with ENCSR488RZS /libraries/ENCLB989CJP/
Problems with ENCSR489KPP /libraries/ENCLB399LDV/
Problems with ENCSR489KPP /libraries/ENCLB399LDV/
Problems with ENCSR489KPP /libraries/ENCLB399LDV/
Problems with ENCSR489KPP /libraries/ENCLB399LDV/
Problems with ENCSR493ZQU /libraries/ENCLB862YMG/
Problems with ENCSR493ZQU /libraries/ENCLB862YMG/
Problems with ENCSR493ZQU /libraries/ENCLB862YMG/
Problems with ENCSR493ZQU /libraries/ENCLB862YMG/
Problems with ENCSR498KBD /libraries/ENCLB188WUM/
Problems with ENCSR498KBD /libraries/ENCLB188WUM/
Problems with ENCSR510HAO /libraries/ENCLB805MAL/
Problems with ENCSR510HAO /libraries/ENCLB805MAL/
Problems with ENCSR510HAO /libraries/ENCLB805MAL/
Problems with ENCSR510HAO /libraries/ENCLB805MAL/
Problems with ENCSR511XOL /libraries/ENCLB460OEF/
Problems with ENCSR511XOL /libraries/ENCLB460OEF/
Problems with ENCSR511XOL /libraries/ENCLB460OEF/
Problems with ENCSR511XOL /libraries/ENCLB460OEF/
Problems with ENCSR519HYQ /libraries/ENCLB369KJQ/
Problems with ENCSR519HYQ /libraries/ENCLB369KJQ/
Problems with ENCSR519HYQ /libraries/ENCLB369KJQ/
Problems with ENCSR519HYQ /libraries/ENCLB369KJQ/
Problems with ENCSR526WJT /libraries/ENCLB656UZY/
Problems with ENCSR526WJT /libraries/ENCLB656UZY/
Problems with ENCSR526WJT /libraries/ENCLB656UZY/
Problems with ENCSR526WJT /libraries/ENCLB656UZY/
Problems with ENCSR536EAF /libraries/ENCLB891WFR/
Problems with ENCSR536EAF /libraries/ENCLB891WFR/
Problems with ENCSR536EAF /libraries/ENCLB891WFR/
Problems with ENCSR536EAF /libraries/ENCLB891WFR/
Problems with ENCSR542LRP /libraries/ENCLB700LWU/
Problems with ENCSR542LRP /libraries/ENCLB700LWU/
Problems with ENCSR555ATU /libraries/ENCLB908IUA/
Problems with ENCSR555ATU /libraries/ENCLB908IUA/
Problems with ENCSR560WNI /libraries/ENCLB354YXN/
Problems with ENCSR560WNI /libraries/ENCLB354YXN/
Problems with ENCSR560WNI /libraries/ENCLB354YXN/
Problems with ENCSR560WNI /libraries/ENCLB354YXN/
Problems with ENCSR561PWI /libraries/ENCLB785FXH/
Problems with ENCSR561PWI /libraries/ENCLB785FXH/
Problems with ENCSR563WPL /libraries/ENCLB782BWD/
Problems with ENCSR563WPL /libraries/ENCLB782BWD/
Problems with ENCSR563WPL /libraries/ENCLB782BWD/
Problems with ENCSR563WPL /libraries/ENCLB782BWD/
Problems with ENCSR566XHU /libraries/ENCLB416NMR/
Problems with ENCSR566XHU /libraries/ENCLB416NMR/
Problems with ENCSR576RBX /libraries/ENCLB902RKO/
Problems with ENCSR576RBX /libraries/ENCLB902RKO/
Problems with ENCSR586LMA /libraries/ENCLB332MGL/
Problems with ENCSR586LMA /libraries/ENCLB332MGL/
Problems with ENCSR586LMA /libraries/ENCLB332MGL/
Problems with ENCSR586LMA /libraries/ENCLB332MGL/
Problems with ENCSR595LVS /libraries/ENCLB678GYJ/
Problems with ENCSR595LVS /libraries/ENCLB678GYJ/
Problems with ENCSR595LVS /libraries/ENCLB678GYJ/
Problems with ENCSR595LVS /libraries/ENCLB678GYJ/
Problems with ENCSR601CQM /libraries/ENCLB524IGV/
Problems with ENCSR601CQM /libraries/ENCLB524IGV/
Problems with ENCSR601CQM /libraries/ENCLB524IGV/
Problems with ENCSR601CQM /libraries/ENCLB524IGV/
Problems with ENCSR635GFL /libraries/ENCLB589IWN/
Problems with ENCSR635GFL /libraries/ENCLB589IWN/
Problems with ENCSR635GFL /libraries/ENCLB589IWN/
Problems with ENCSR635GFL /libraries/ENCLB589IWN/
Problems with ENCSR635GRA /libraries/ENCLB250XVZ/
Problems with ENCSR635GRA /libraries/ENCLB250XVZ/
Problems with ENCSR635GRA /libraries/ENCLB250XVZ/
Problems with ENCSR635GRA /libraries/ENCLB250XVZ/
Problems with ENCSR637AZA /libraries/ENCLB581NBT/
Problems with ENCSR637AZA /libraries/ENCLB581NBT/
Problems with ENCSR637AZA /libraries/ENCLB581NBT/
Problems with ENCSR637AZA /libraries/ENCLB581NBT/
Problems with ENCSR642TKQ /libraries/ENCLB765ZDM/
Problems with ENCSR642TKQ /libraries/ENCLB765ZDM/
Problems with ENCSR644EUQ /libraries/ENCLB146JLP/
Problems with ENCSR644EUQ /libraries/ENCLB146JLP/
Problems with ENCSR644YWO /libraries/ENCLB517TXV/
Problems with ENCSR644YWO /libraries/ENCLB517TXV/
Problems with ENCSR644YWO /libraries/ENCLB517TXV/
Problems with ENCSR644YWO /libraries/ENCLB517TXV/
Problems with ENCSR651QGH /libraries/ENCLB308YIM/
Problems with ENCSR651QGH /libraries/ENCLB308YIM/
Problems with ENCSR651QGH /libraries/ENCLB308YIM/
Problems with ENCSR651QGH /libraries/ENCLB308YIM/
Problems with ENCSR671RJG /libraries/ENCLB885OKL/
Problems with ENCSR671RJG /libraries/ENCLB885OKL/
Problems with ENCSR671RJG /libraries/ENCLB885OKL/
Problems with ENCSR671RJG /libraries/ENCLB885OKL/
Problems with ENCSR692OZP /libraries/ENCLB194LYE/
Problems with ENCSR692OZP /libraries/ENCLB194LYE/
Problems with ENCSR692OZP /libraries/ENCLB194LYE/
Problems with ENCSR692OZP /libraries/ENCLB194LYE/
Problems with ENCSR692ZWB /libraries/ENCLB598TEK/
Problems with ENCSR692ZWB /libraries/ENCLB598TEK/
Problems with ENCSR692ZWB /libraries/ENCLB598TEK/
Problems with ENCSR692ZWB /libraries/ENCLB598TEK/
Problems with ENCSR694DQP /libraries/ENCLB170MZP/
Problems with ENCSR694DQP /libraries/ENCLB170MZP/
Problems with ENCSR694DQP /libraries/ENCLB170MZP/
Problems with ENCSR694DQP /libraries/ENCLB170MZP/
Problems with ENCSR732MDS /libraries/ENCLB701VGG/
Problems with ENCSR732MDS /libraries/ENCLB701VGG/
Problems with ENCSR732MDS /libraries/ENCLB701VGG/
Problems with ENCSR732MDS /libraries/ENCLB701VGG/
Problems with ENCSR745UMG /libraries/ENCLB709WAG/
Problems with ENCSR745UMG /libraries/ENCLB709WAG/
Problems with ENCSR773FPF /libraries/ENCLB536TLF/
Problems with ENCSR773FPF /libraries/ENCLB536TLF/
Problems with ENCSR773FPF /libraries/ENCLB536TLF/
Problems with ENCSR773FPF /libraries/ENCLB536TLF/
Problems with ENCSR774KTU /libraries/ENCLB727KBM/
Problems with ENCSR774KTU /libraries/ENCLB727KBM/
Problems with ENCSR774KTU /libraries/ENCLB727KBM/
Problems with ENCSR774KTU /libraries/ENCLB727KBM/
Problems with ENCSR799GLH /libraries/ENCLB899ORD/
Problems with ENCSR799GLH /libraries/ENCLB899ORD/
Problems with ENCSR799GLH /libraries/ENCLB899ORD/
Problems with ENCSR799GLH /libraries/ENCLB899ORD/
Problems with ENCSR801MJO /libraries/ENCLB449BXX/
Problems with ENCSR801MJO /libraries/ENCLB449BXX/
Problems with ENCSR801MJO /libraries/ENCLB449BXX/
Problems with ENCSR801MJO /libraries/ENCLB449BXX/
Problems with ENCSR813ULV /libraries/ENCLB963QFG/
Problems with ENCSR813ULV /libraries/ENCLB963QFG/
Problems with ENCSR813ULV /libraries/ENCLB963QFG/
Problems with ENCSR813ULV /libraries/ENCLB963QFG/
Problems with ENCSR827YHY /libraries/ENCLB389BSV/
Problems with ENCSR827YHY /libraries/ENCLB389BSV/
Problems with ENCSR827YHY /libraries/ENCLB389BSV/
Problems with ENCSR827YHY /libraries/ENCLB389BSV/
Problems with ENCSR828TLP /libraries/ENCLB611DGC/
Problems with ENCSR828TLP /libraries/ENCLB611DGC/
Problems with ENCSR828TLP /libraries/ENCLB611DGC/
Problems with ENCSR828TLP /libraries/ENCLB611DGC/
Problems with ENCSR830MSX /libraries/ENCLB231PCO/
Problems with ENCSR830MSX /libraries/ENCLB231PCO/
Problems with ENCSR830MSX /libraries/ENCLB231PCO/
Problems with ENCSR830MSX /libraries/ENCLB231PCO/
Problems with ENCSR834SCG /libraries/ENCLB849DVZ/
Problems with ENCSR834SCG /libraries/ENCLB849DVZ/
Problems with ENCSR834SCG /libraries/ENCLB849DVZ/
Problems with ENCSR834SCG /libraries/ENCLB849DVZ/
Problems with ENCSR847STS /libraries/ENCLB735WRJ/
Problems with ENCSR847STS /libraries/ENCLB735WRJ/
Problems with ENCSR847STS /libraries/ENCLB735WRJ/
Problems with ENCSR847STS /libraries/ENCLB735WRJ/
Problems with ENCSR851RNW /libraries/ENCLB349QMF/
Problems with ENCSR851RNW /libraries/ENCLB349QMF/
Problems with ENCSR851RNW /libraries/ENCLB349QMF/
Problems with ENCSR851RNW /libraries/ENCLB349QMF/
Problems with ENCSR852NQO /libraries/ENCLB299EAG/
Problems with ENCSR852NQO /libraries/ENCLB299EAG/
Problems with ENCSR852NQO /libraries/ENCLB299EAG/
Problems with ENCSR852NQO /libraries/ENCLB299EAG/
Problems with ENCSR855ZYP /libraries/ENCLB193HSH/
Problems with ENCSR855ZYP /libraries/ENCLB193HSH/
Problems with ENCSR855ZYP /libraries/ENCLB193HSH/
Problems with ENCSR855ZYP /libraries/ENCLB193HSH/
Problems with ENCSR877CFF /libraries/ENCLB227FXC/
Problems with ENCSR877CFF /libraries/ENCLB227FXC/
Problems with ENCSR877CFF /libraries/ENCLB227FXC/
Problems with ENCSR877CFF /libraries/ENCLB227FXC/
Problems with ENCSR877UAI /libraries/ENCLB747XAC/
Problems with ENCSR877UAI /libraries/ENCLB747XAC/
Problems with ENCSR877UAI /libraries/ENCLB747XAC/
Problems with ENCSR877UAI /libraries/ENCLB747XAC/
Problems with ENCSR880DYE /libraries/ENCLB977UQY/
Problems with ENCSR880DYE /libraries/ENCLB977UQY/
Problems with ENCSR880DYE /libraries/ENCLB977UQY/
Problems with ENCSR880DYE /libraries/ENCLB977UQY/
Problems with ENCSR880MVS /libraries/ENCLB586GGX/
Problems with ENCSR880MVS /libraries/ENCLB586GGX/
Problems with ENCSR880MVS /libraries/ENCLB586GGX/
Problems with ENCSR880MVS /libraries/ENCLB586GGX/
Problems with ENCSR888NNP /libraries/ENCLB639GHE/
Problems with ENCSR888NNP /libraries/ENCLB639GHE/
Problems with ENCSR888NNP /libraries/ENCLB639GHE/
Problems with ENCSR888NNP /libraries/ENCLB639GHE/
Problems with ENCSR894JVL /libraries/ENCLB868TVO/
Problems with ENCSR894JVL /libraries/ENCLB868TVO/
Problems with ENCSR894JVL /libraries/ENCLB868TVO/
Problems with ENCSR894JVL /libraries/ENCLB868TVO/
Problems with ENCSR905YQE /libraries/ENCLB822SVY/
Problems with ENCSR905YQE /libraries/ENCLB822SVY/
Problems with ENCSR905YQE /libraries/ENCLB822SVY/
Problems with ENCSR905YQE /libraries/ENCLB822SVY/
Problems with ENCSR912IHV /libraries/ENCLB849GNT/
Problems with ENCSR912IHV /libraries/ENCLB849GNT/
Problems with ENCSR912IHV /libraries/ENCLB849GNT/
Problems with ENCSR912IHV /libraries/ENCLB849GNT/
Problems with ENCSR915UNZ /libraries/ENCLB151ZAG/
Problems with ENCSR915UNZ /libraries/ENCLB151ZAG/
Problems with ENCSR915UNZ /libraries/ENCLB151ZAG/
Problems with ENCSR915UNZ /libraries/ENCLB151ZAG/
Problems with ENCSR923NSW /libraries/ENCLB143DFU/
Problems with ENCSR923NSW /libraries/ENCLB143DFU/
Problems with ENCSR923NSW /libraries/ENCLB143DFU/
Problems with ENCSR923NSW /libraries/ENCLB143DFU/
Problems with ENCSR929PEE /libraries/ENCLB959AIF/
Problems with ENCSR929PEE /libraries/ENCLB959AIF/
Problems with ENCSR932DTM /libraries/ENCLB905AUU/
Problems with ENCSR932DTM /libraries/ENCLB905AUU/
Problems with ENCSR932DTM /libraries/ENCLB905AUU/
Problems with ENCSR932DTM /libraries/ENCLB905AUU/
Problems with ENCSR948XEM /libraries/ENCLB315FQV/
Problems with ENCSR948XEM /libraries/ENCLB315FQV/
Problems with ENCSR962DRE /libraries/ENCLB697JVD/
Problems with ENCSR962DRE /libraries/ENCLB697JVD/
Problems with ENCSR962DRE /libraries/ENCLB697JVD/
Problems with ENCSR962DRE /libraries/ENCLB697JVD/
Problems with ENCSR969VQQ /libraries/ENCLB170FEN/
Problems with ENCSR969VQQ /libraries/ENCLB170FEN/
Problems with ENCSR972KJU /libraries/ENCLB762GOP/
Problems with ENCSR972KJU /libraries/ENCLB762GOP/
Problems with ENCSR972KJU /libraries/ENCLB762GOP/
Problems with ENCSR972KJU /libraries/ENCLB762GOP/
Problems with ENCSR973ZJO /libraries/ENCLB374JXM/
Problems with ENCSR973ZJO /libraries/ENCLB374JXM/
Problems with ENCSR973ZJO /libraries/ENCLB374JXM/
Problems with ENCSR973ZJO /libraries/ENCLB374JXM/
Problems with ENCSR974VTR /libraries/ENCLB415PDV/
Problems with ENCSR974VTR /libraries/ENCLB415PDV/
Problems with ENCSR974VTR /libraries/ENCLB415PDV/
Problems with ENCSR974VTR /libraries/ENCLB415PDV/
Problems with ENCSR976FIL /libraries/ENCLB124EIT/
Problems with ENCSR976FIL /libraries/ENCLB124EIT/
Problems with ENCSR976FIL /libraries/ENCLB124EIT/
Problems with ENCSR976FIL /libraries/ENCLB124EIT/
Problems with ENCSR983WDT /libraries/ENCLB708YOS/
Problems with ENCSR983WDT /libraries/ENCLB708YOS/
Problems with ENCSR983WDT /libraries/ENCLB708YOS/
Problems with ENCSR983WDT /libraries/ENCLB708YOS/
Problems with ENCSR985FZQ /libraries/ENCLB292IDD/
Problems with ENCSR985FZQ /libraries/ENCLB292IDD/
Problems with ENCSR985FZQ /libraries/ENCLB292IDD/
Problems with ENCSR985FZQ /libraries/ENCLB292IDD/
Problems with ENCSR986QTH /libraries/ENCLB571XKG/
Problems with ENCSR986QTH /libraries/ENCLB571XKG/
Problems with ENCSR109DAN /libraries/ENCLB527JDU/
Problems with ENCSR109DAN /libraries/ENCLB527JDU/
Problems with ENCSR388EIN /libraries/ENCLB952MZJ/
Problems with ENCSR388EIN /libraries/ENCLB952MZJ/
Problems with ENCSR452OYH /libraries/ENCLB294FBZ/
Problems with ENCSR452OYH /libraries/ENCLB294FBZ/
Problems with ENCSR503QZJ /libraries/ENCLB129LMS/
Problems with ENCSR503QZJ /libraries/ENCLB129LMS/
Problems with ENCSR572SMD /libraries/ENCLB273ZWE/
Problems with ENCSR572SMD /libraries/ENCLB273ZWE/
Problems with ENCSR877NAX /libraries/ENCLB514RKT/
Problems with ENCSR877NAX /libraries/ENCLB514RKT/""".split("\n"))

In [ ]:
len(problems)

In [ ]:
problem_experiments = []
for row in problems:
    problem_experiments.append(row.split()[2])
problem_experiments

In [ ]:
processed = set()
processed.update(experiments_to_run_2_6)
processed.update(experiments_to_run_7_21)
processed.update(experiments_to_run_22_51)
processed.update(experiments_to_run_52_81)
processed.update(experiments_to_run_82_107)

In [ ]:
processed.intersection(problem_experiments)